# Pytorch CNN to Test on the Generated Samples

<img src='https://miro.medium.com/max/3288/1*uAeANQIOQPqWZnnuH-VEyw.jpeg'   width="700" height="350">

#### Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
os.chdir('c:/users/nicolas/documents/data/faces')

#### Loading all file names for the TRAIN set

In [2]:
files = glob('combined/*.jpg')

#### Getting labels 

In [3]:
faces = [i for i in files if (i[-34] in ('0', '1')) and len(i[-37:-35].strip('\\').strip('d'))  == 2 ]

In [4]:
y = [i[-34] for i in files if (i[-34] in ('0', '1')) and len(i[-37:-35].strip('\\').strip('d')) > 1 ]

In [5]:
sex = ['men', 'women']

In [6]:
assert len(y) == len(faces), 'The X and Y are not of the same length!'

#### Getting shape info

In [7]:
nrow, ncol, nchan = 60, 60, 3

#### This is the shape width/height

In [8]:
dim = 60

#### Cropping function

In [9]:
def crop(img):
    if img.shape[0]<img.shape[1]:
        x = img.shape[0]
        y = img.shape[1]
        crop_img = img[: , int(y/2-x/2):int(y/2+x/2)]
    else:
        x = img.shape[1]
        y = img.shape[0]
        crop_img = img[int(y/2-x/2):int(y/2+x/2) , :]

    return crop_img

##### Loading and cropping images

In [10]:
print('Scaling...', end='')
start = time()
x = []
num_to_load = len(faces)
for ix, file in enumerate(faces[:num_to_load]): 
    image = plt.imread(file, 'jpg')
    image = Image.fromarray(image).resize((dim, dim)).convert('L')
    image = crop(np.array(image))
    x.append(image)
print(f'\rDone. {int(time() - start)} seconds')
y = y[:num_to_load]

Done. 15 seconds


##### Turning the pictures into arrays

In [11]:
xtrain = np.array(x, dtype=np.float32)
ytrain = np.array(y, dtype=np.float32)

In [12]:
assert xtrain.shape[1] == dim
print(xtrain.shape)

(20638, 60, 60)


In [13]:
if xtrain.shape[0] == ytrain.shape[0]:
    print('X and Y shapes are correct! (%i samples each)' % xtrain.shape[0])

X and Y shapes are correct! (20638 samples each)


In [14]:
assert xtrain.ndim == 3

In [15]:
print(f'The size of the data we are using is {xtrain.shape[0]:,} pictures.')

The size of the data we are using is 20,638 pictures.


In [16]:
files, faces = None, None

#### Cross-validation, splitting input

In [17]:
men = np.random.choice(np.where(ytrain == 0)[0], 8_000)
m_array = xtrain[men]
m_target = ytrain[men]

In [18]:
women = np.random.choice(np.where(ytrain == 1)[0], 8_000)
w_array = xtrain[women]
w_target = ytrain[women]

In [19]:
x_train = np.vstack([m_array, w_array])
y_train = np.concatenate([m_target, w_target], axis=0).astype(np.int32)

In [20]:
x_train.shape, y_train.shape

((16000, 60, 60), (16000,))

In [21]:
permut = np.random.permutation(np.arange(x_train.shape[0]))
x_train = x_train[permut]
y_train = y_train[permut]
assert y_train.sum() == 8_000, 'The classes aren\'t balanced.'

In [22]:
x_size, y_size = x_train.shape[0], y_train.shape[0]
print(f'The size of X is {x_size:,} and the '\
     f'size of Y is {y_size:,}.')

The size of X is 16,000 and the size of Y is 16,000.


#### Getting the test set for MEN

In [23]:
directory = 'generated_men_aae'

In [24]:
test_files = glob(directory + '/split' + '/*.png')

In [25]:
assert len(test_files) == 25_000, 'There is not 25,000 files, '\
    'but %i.' % len(test_files)

In [26]:
print('Scaling...', end='')
start = time()
x = []
num_to_load = 10_000
for ix, file in enumerate(test_files[:num_to_load]): 
    image = plt.imread(file, 'jpg')
    image = Image.fromarray(image).resize((dim, dim)).convert('L')
    image = np.array(image)
    x.append(image)
print(f'\rDone. {int(time() - start)} seconds')
y_gen_men = np.repeat(0, num_to_load)

Done. 4 seconds


In [27]:
assert np.array(x).shape == (num_to_load, 60, 60)

In [28]:
start = time()
xtest_men = np.array(x, dtype=np.float32)
ytest_men = np.array(y_gen_men, dtype=np.int32)
print(f'\rDone. {int(time() - start)} seconds')

Done. 0 seconds


#### Getting the test set for WOMEN

In [29]:
directory = 'generated_women_aae'

In [30]:
test_files = glob(directory + '/split' + '/*.png')

In [31]:
assert len(test_files) == 25_000, 'There is not 25,000 files, '\
    'but %i.' % len(test_files)

In [32]:
print('Scaling...', end='')
start = time()
x = []
num_to_load = 10_000
for ix, file in enumerate(test_files[:num_to_load]): 
    image = plt.imread(file, 'jpg')
    image = Image.fromarray(image).resize((dim, dim)).convert('L')
    image = np.array(image)
    x.append(image)
print(f'\rDone. {int(time() - start)} seconds')
y_gen_women = np.repeat(1, num_to_load)

Done. 5 seconds


In [33]:
xtest_women = np.array(x, dtype=np.float32)
ytest_women = np.array(y_gen_women, dtype=np.int32)

In [34]:
assert np.array(x).shape == (num_to_load, 60, 60)

#### Merging the test set

In [35]:
x_test = np.vstack([xtest_men, xtest_women])
y_test = np.concatenate([ytest_men, ytest_women], axis=0)

In [36]:
x_train.shape, y_train.shape

((16000, 60, 60), (16000,))

In [37]:
permut = np.random.permutation(np.arange(x_test.shape[0]))
x_test = x_test[permut]
y_test = y_test[permut]
assert y_test.sum() == y_test.shape[0]/2, 'The classes aren\'t balanced.'

In [38]:
x_size, y_size = x_test.shape[0], y_test.shape[0]
print(f'The size of X is {x_size:,} and the '\
     f'size of Y is {y_size:,}.')

The size of X is 20,000 and the size of Y is 20,000.


In [39]:
assert x_test.shape[0] == x_test.shape[0] == y_test.shape[0] == y_test.shape[0]

#### One hot encoding the targets

In [40]:
# y_train = np.eye(2)[y_train]
# y_test = np.eye(2)[y_test]

#### Scaling, casting the arrays

In [41]:
print('Scaling...', end='') 
x_train = x_train.reshape(-1, 1, dim, dim).astype('float32') / 255 
x_test = x_test.reshape(-1, 1, dim, dim).astype('float32') / 255
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')
print('\rDone.     ')

Done.     


In [42]:
samples, first, second, third = x_test.shape
print('First dimension: %i' % samples,
     '\nSecond dimension: %i' % first,
     '\nThird dimension: %i' % second,
     '\nFourth dimension: %i' % third)

First dimension: 20000 
Second dimension: 1 
Third dimension: 60 
Fourth dimension: 60


#### Sending the arrays to Cuda

In [43]:
if torch.cuda.is_available():
    x_train = torch.from_numpy(x_train) 
    x_test = torch.from_numpy(x_test) 
    y_train = torch.from_numpy(y_train) 
    y_test = torch.from_numpy(y_test)
    print('Tensors successfully flushed to CUDA.')
else:
    print('CUDA not available!')

Tensors successfully flushed to CUDA.


##### Clearning memory

In [44]:
x, y = None, None

##### Building the ConvNet

Initially image size, W = 60 <br>
Kernel Size, k = 3 <br>
Stride , s = 1 <br>
Padding, P = 0 <br>
The formula for the number of outputs to the next layer of conv2d is: O = { (W - k + 2*P)/s } + 1

In [45]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        a = 32 #* 4
        b = 64 #* 4
        c = 128 #* 4
        self.conv1 = nn.Conv2d(1, a, 3)
        self.conv2 = nn.Conv2d(a, b, 3)
        self.conv3 = nn.Conv2d(b, c, 3)
        
        self.fc1 = nn.Linear(5*5*c, 1024) 
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 2)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)
        return x

In [46]:
net = ConvNet()

In [47]:
if torch.cuda.is_available():
    net.cuda()

In [48]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [49]:
loss_function = nn.CrossEntropyLoss()

##### Instantiating the data

In [50]:
class FaceTrain:
    
    def __init__(self):
        self.len = x_train.shape[0]
        self.x_train = x_train
        self.y_train = y_train
        
    def __getitem__(self, index):
        return x_train[index], y_train[index]#.unsqueeze(0)
    
    def __len__(self):
        return self.len

In [51]:
class FaceTest:
    
    def __init__(self):
        self.len = x_test.shape[0]
        self.x_test = x_test
        self.y_test = y_test
        
    def __getitem__(self, index):
        return x_test[index], y_test[index]#.unsqueeze(0)
    
    def __len__(self):
        return self.len

##### Making instances of the data

In [52]:
train = FaceTrain()
test = FaceTest()

##### Making data iterator

In [53]:
train_loader = DataLoader(dataset=train, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test, batch_size=64, shuffle=True)

##### Training the model

In [55]:
epochs = 30
steps = 0
train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    net.train()
    for images, labels in train_loader:   
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()     
        optimizer.zero_grad()
        log_ps = net(images)
        loss = loss_function(log_ps, labels)
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()        
    else:
        test_loss = 0
        accuracy = 0        
        net.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                if torch.cuda.is_available():
                    images, labels = images.cuda(), labels.cuda()
                log_ps = net(images)
                test_loss += loss_function(log_ps, labels)                
                # ps = torch.exp(log_ps)
                top_p, top_class = log_ps.topk(1, dim=1)
                equals = top_class.long() == labels.long().view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))                
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))
        print("[Epoch: {}/{}] ".format(e+1, epochs),
              "[Training Loss: {:.3f}] ".format(running_loss/len(train_loader)),
              "[Test Loss: {:.3f}] ".format(test_loss/len(test_loader)),
              "[Test Accuracy: {:.3f}]".format(accuracy/len(test_loader)))

[Epoch: 1/30]  [Training Loss: 0.194]  [Test Loss: 0.043]  [Test Accuracy: 0.997]
[Epoch: 2/30]  [Training Loss: 0.175]  [Test Loss: 0.016]  [Test Accuracy: 0.999]
[Epoch: 3/30]  [Training Loss: 0.152]  [Test Loss: 0.017]  [Test Accuracy: 0.999]
[Epoch: 4/30]  [Training Loss: 0.132]  [Test Loss: 0.013]  [Test Accuracy: 1.000]
[Epoch: 5/30]  [Training Loss: 0.120]  [Test Loss: 0.023]  [Test Accuracy: 1.000]
[Epoch: 6/30]  [Training Loss: 0.102]  [Test Loss: 0.028]  [Test Accuracy: 0.999]
[Epoch: 7/30]  [Training Loss: 0.099]  [Test Loss: 0.010]  [Test Accuracy: 1.000]
[Epoch: 8/30]  [Training Loss: 0.084]  [Test Loss: 0.012]  [Test Accuracy: 0.999]
[Epoch: 9/30]  [Training Loss: 0.067]  [Test Loss: 0.010]  [Test Accuracy: 1.000]
[Epoch: 10/30]  [Training Loss: 0.061]  [Test Loss: 0.008]  [Test Accuracy: 1.000]
[Epoch: 11/30]  [Training Loss: 0.054]  [Test Loss: 0.012]  [Test Accuracy: 0.997]
[Epoch: 12/30]  [Training Loss: 0.050]  [Test Loss: 0.003]  [Test Accuracy: 1.000]
[Epoch: 13/30